In [45]:
# Activate Bokeh Environment

from bokeh.plotting import figure,show
from bokeh.io import output_notebook, push_notebook, curdoc, reset_output
from bokeh.models import ColumnDataSource, HoverTool, FactorRange, NumeralTickFormatter, Select, Slider, RangeTool, Range1d
from bokeh.layouts import gridplot, column, row, layout
from bokeh.transform import factor_cmap
from bokeh.models.annotations import Label
from jupyter_bokeh import BokehModel
from bokeh.palettes import d3

from ipywidgets import interact

import pandas as pd
import numpy as np

In [2]:
# activate Bokeh output in Jupyter notebook
output_notebook()

Loading BokehJS ...

In [6]:
df4 = pd.read_excel("data/data.xlsx")
df4

,Name,House property price,Gold price,Fixed term deposit rate,Shanghai stock composite index,CPI,Unnamed: 6,inflation rate,Unnamed: 8,government bond yield,Unnamed: 10,Unnamed: 11,second-had car
0,Unit,CNY/m^2,元/克,%,point,core CPI,headline CPI,core inflation,headline inflation,6-month,5-year,10-year,NaN
1,来源,Wind,上海黄金交易所,中国人民银行,上海证券交易所,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2010-03-31 00:00:00,6399,243.91,4.15,3109.105,84.476667,77.396667,0.00273,0.019406,0.014136,0.027923,0.034818,NaN
3,2010-06-30 00:00:00,6657,271.6,4.15,2398.37,84.753333,77.286667,0.003275,-0.001421,0.021017,0.023997,0.03279,NaN
4,2010-09-30 00:00:00,7049,282.35,4.15,2655.658,84.993333,77.723333,0.002832,0.00565,0.018884,0.023473,0.033183,NaN
5,2010-12-31 00:00:00,7492,301.11,4.15,2808.077,85.586667,79.446667,0.006981,0.022173,0.02358,0.025526,0.027141,NaN
6,2011-03-31 00:00:00,7845,302.07,4.5,2928.111,86.203333,81.393333,0.007205,0.024503,0.027008,0.032328,0.03912,NaN
7,2011-06-30 00:00:00,7779,314.32,4.75,2762.076,86.68,81.826667,0.00553,0.005324,0.034127,0.0347,0.038957,NaN
8,2011-09-30 00:00:00,7542,338.08,5,2359.22,86.956667,82.73,0.003192,0.01104,0.034729,0.037119,0.038605,NaN
9,2011-12-31 00:00:00,7026,319.49,5,2199.417,87.153333,83.14,0.002262,0.004956,0.026641,0.030376,0.034303,NaN


In [7]:
df4=df4.drop(index=df4.index[:2])

In [8]:
df4=df4.rename(columns={'Name': 'Date', 'Unnamed: 6': 'Headline CPI', 'CPI': 'Core CPI', 'Unnamed: 8':'Headline Inflation', 'inflation rate':'Core Inflation','Unnamed: 10':'Government Bond 5-year','Unnamed: 11':'Government Bond 10-year', 'government bond yield':'Government Bond 6-month'})

In [10]:
df4['Date']=pd.to_datetime(df4['Date'])
df4['Year']=df4['Date'].dt.year
df4['Quarter']=df4['Date'].dt.quarter
df4['Quarter'] = 'Q' + df4['Quarter'].astype(str)

In [12]:
df_asset_subset = df4.drop(columns=['Date','second-had car','Year','Quarter','Core CPI','Headline CPI','Core Inflation','Headline Inflation'])
#asset = df_asset_subset.columns.tolist()

In [11]:
df_asset=df4[['Year','Quarter','House property price']].copy()
df_asset.iloc[-1] = df_asset.iloc[-1].fillna(df_asset.iloc[-2])
df_asset['House pct']=df_asset['House property price'].pct_change() * 100
df_asset['Gold price']=df4[['Gold price']].copy()
df_asset['Gold pct']=df_asset['Gold price'].pct_change() * 100
df_asset['Fixed term deposit rate']=df4[['Fixed term deposit rate']].copy()
df_asset['Deposit pct']=df_asset['Fixed term deposit rate'].pct_change() * 100
df_asset['Shanghai stock composite index']=df4[['Shanghai stock composite index']].copy()
df_asset['Stock pct']=df_asset['Shanghai stock composite index'].pct_change() * 100
df_asset['Government Bond 6-month']=df4[['Government Bond 6-month']].copy()
df_asset['Bond 6 month pct']=df_asset['Government Bond 6-month'].pct_change() * 100
df_asset['Government Bond 5-year']=df4[['Government Bond 5-year']].copy()
df_asset['Bond 5 year pct']=df_asset['Government Bond 5-year'].pct_change() * 100
df_asset['Government Bond 10-year']=df4[['Government Bond 10-year']].copy()
df_asset['Bond 10 year pct']=df_asset['Government Bond 10-year'].pct_change() * 100

In [12]:
pct_date = ['Year','Quarter']
pct_columns= [col for col in df_asset.columns if 'pct' in col]
pct = pct_date + pct_columns
df_asset_pct = df_asset[pct]
df_asset_pct.fillna(0, inplace=True)

C:\Users\darleneQ\AppData\Local\Temp\ipykernel_25312\1291750124.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_asset_pct.fillna(0, inplace=True)


In [13]:
df = df_asset_pct.drop(columns=['Year','Quarter'])
asset_pct = df.columns.tolist()
#asset_pct

In [14]:
df_inflation=df4[['Year','Quarter','Core CPI','Headline CPI','Core Inflation','Headline Inflation']].copy()
df_inflation.iloc[-1] = df_inflation.iloc[-1].fillna(df_inflation.iloc[-2])
#df_inflation

In [15]:
df_inflation['Core CPI pct']=df_inflation['Core CPI'].pct_change() * 100
df_inflation['Headline CPI Cpct']=df_inflation['Headline CPI'].pct_change()  * 100
df_inflation['Core Inflation pct'] = df_inflation['Core Inflation'].pct_change()  * 100
df_inflation['Headline Inflation pct']=df_inflation['Headline Inflation'].pct_change() * 100
#df_inflation

In [16]:
pct_column= [col for col in df_inflation.columns if 'pct' in col]
pct2 = pct_date + pct_column
df_inflation_pct = df_inflation[pct2]
df_inflation_pct.fillna(0, inplace=True)
#df_inflation_pct

C:\Users\darleneQ\AppData\Local\Temp\ipykernel_25312\1632322031.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_inflation_pct.fillna(0, inplace=True)


In [17]:
df_in = df_inflation_pct.drop(columns=['Year','Quarter'])
inflation_pct = df_in.columns.tolist()
#inflation_pct

In [18]:
years = sorted(df_asset['Year'].unique().tolist())
quarters = sorted(df_asset['Quarter'].unique().tolist())

In [21]:
select_pct_tag = 'All asset'
select_inflation_tag = 'All inflation'

In [19]:
palette = list(d3['Category20'][12])
all_labels = sorted(set(df_asset_pct.columns) | set(df_inflation_pct.columns))
color_mapping = {label: palette[i % len(palette)] for i, label in enumerate(all_labels)}

In [20]:
def data_prepare_asset(select_tag):
    asset = []
    if select_tag == 'All asset':
        asset = asset_pct
    else:
        asset = [select_tag]
    data = df_asset_pct.pivot_table(
        values=asset,
        index=['Year', 'Quarter']
    )
    
    x_range = FactorRange(factors=[(str(year), quarter, item) for year in years for quarter in quarters for item in asset])
    
    x_labels = [(str(year), quarter, item) for year in years for quarter in quarters for item in asset]
     
    return data, x_labels, x_range

In [21]:
def data_prepare_inflation(select_tag):
    asset = []
    if select_tag == 'All inflation':
        asset = inflation_pct
    else:
        asset = [select_tag]
    data = df_inflation_pct.pivot_table(
        values=asset,
        index=['Year', 'Quarter']
    )
    
    x_range = FactorRange(factors=[(str(year), quarter, item) for year in years for quarter in quarters for item in asset])
    
    x_labels = [(str(year), quarter, item) for year in years for quarter in quarters for item in asset]
     
    return data, x_labels, x_range

In [22]:
def create_source(data, x_labels):
    
    y = data.values.flatten().tolist()
    
    labels = data.columns.tolist() * len(data)
    
    colors = [color_mapping[label] for label in labels]
    
    source = dict(
        x_labels=x_labels,
        y=y,
        label=labels,
        colors=colors,
        Year=[label[0] for label in x_labels],
        Quarter=[label[1] for label in x_labels]
    )
    
    return ColumnDataSource(source)

In [23]:
options = dict(width = 1000, height = 600)

def draw_bar_chart(source, x_range):
    
    p = figure(
        
        x_range = x_range,
        title = 'Asset PCT',
        tools='pan,wheel_zoom,box_zoom,reset,save',
        **options
    )
    
    p.xgrid.grid_line_color = None
    
    p.x_range.range_padding = 0.1
    
    p.xaxis.major_label_text_font_size = '0px'
    
    p.xaxis.major_tick_line_color = None
    
    p.yaxis.axis_label = 'PCT (%)'
    
    p.yaxis.formatter = NumeralTickFormatter(format='0,0')
    
    p.vbar(
        x = 'x_labels',
        top = 'y',
        width = 0.9,
        source = source,
        
        legend_group = 'label',
        line_color = None,
        
        fill_color='colors'
    )
        
    p.add_tools(HoverTool(tooltips=[
            ('Year', '@Year'),
            ('Quarter', '@Quarter'),
            ('Asset', '@label'),
            ('pct value', '@y{0,0.00}')
        ]))
    
    p.legend.label_text_font_size = '7pt'
    p.legend.label_height = 15
    p.legend.glyph_height = 10
    p.legend.glyph_width = 10
    p.legend.orientation = 'vertical'
    p.legend.location = 'top_right'
    
    p.output_backend = "svg"
    
    return p

In [33]:
def plot_pct(doc):
    select_pct_tag = 'All asset'
    select_inflation_tag = 'All inflation'
    
    data, x_labels, x_range = data_prepare_asset(select_pct_tag)
    source = create_source(data, x_labels)
    data2, x_labels2, x_range2 = data_prepare_inflation(select_inflation_tag)
    source2 = create_source(data2, x_labels2)

    p = draw_bar_chart(source, x_range)
    h = draw_bar_chart(source2, x_range2)
    
    select_opt = ['All asset'] + asset_pct
    inflation_opt = ['All inflation'] + inflation_pct
    select_asset = Select(title="Select Asset:", value=select_opt[0], options=select_opt)
    select_inflation = Select(title="Select Inflation:", value=inflation_opt[0], options=inflation_opt)

    def update_asset_chart(attr, old, new):
        #global select_pct_tag, source
        select_pct_tag = new
        data, labels, range = data_prepare_asset(select_pct_tag)
        source = create_source(data, labels)
        layout1.children[1].children[0] = draw_bar_chart(source, range)

    def update_inflation_chart(attr, old, new):
        #global select_inflation_tag, source2
        select_inflation_tag = new
        data2, labels2, range2 = data_prepare_inflation(select_inflation_tag)
        source2 = create_source(data2, labels2)
        layout2.children[1].children[0] = draw_bar_chart(source2, range2)
    
    select_asset.on_change('value', update_asset_chart)
    select_inflation.on_change('value', update_inflation_chart)
    
    layout1 = column(column(select_asset), column(p))
    layout2 = column(column(select_inflation), column(h))
    
    doc.add_root(column(layout1,layout2))

In [35]:
show(plot_pct, notebook_url="http://localhost:8888")

In [58]:
def create_source_proportion(asset_tag, inflation_tag):
    merged_df = pd.merge(df_asset_pct, df_inflation_pct, on=['Year', 'Quarter'], how='inner')
    merged_df['Proportion'] = merged_df[asset_tag] / merged_df[inflation_tag].replace({0: np.nan})
    merged_df['Datetime'] = merged_df['Year'].astype(str) + merged_df['Quarter']
    data = merged_df[['Datetime', 'Proportion']].copy()
    data['label'] = 'The proportion of ' + asset_tag + ' and ' + inflation_tag

    return ColumnDataSource(data)

In [88]:
def draw_line_chart(source):
    p = figure(
        width = 1000,
        height = 800,
        title = 'Proportion',
        x_range = source.data['Datetime'],
        tools = "pan,wheel_zoom,box_zoom,reset,save",
        toolbar_location = 'right'
    )

    p.xgrid.grid_line_color = None
    p.xaxis.major_label_orientation = 'vertical'

    p.line(
        x = 'Datetime',
        y = 'Proportion',
        source = source,
        color = '#999999'
    )

    p.add_tools(HoverTool(tooltips=[('Time', '@Datetime'), ('Proportion', '@Proportion')]))

    label = Label(
        x=700,
        y=650,
        x_units = 'screen',
        y_units = 'screen',
        text = source.data['label'][0],
        text_font_size = '10pt',
        text_font_style = 'bold', 
        text_color = 'black',
        text_align = 'center'
    )
    p.add_layout(label)
        
    p.output_backend = 'svg'

    return p

In [97]:
def plot_proportion(doc):
    asset_tag = 'House pct'
    inflation_tag = 'Core CPI pct'

    source = create_source_proportion(asset_tag, inflation_tag)

    p = draw_line_chart(source)

    select_asset2 = Select(title="Select Asset:", value=asset_pct[0], options=asset_pct)
    select_inflation2 = Select(title="Select Inflation:", value=inflation_pct[0], options=inflation_pct)

    def update_asset(attr, old, new):
        asset_tag = new
        source = create_source_proportion(asset_tag, inflation_tag)
        layout2.children[0] = draw_line_chart(source)

    def update_inflation(attr, old, new):
        inflation_tag = new
        source = create_source_proportion(asset_tag, inflation_tag)
        layout2.children[0] = draw_line_chart(source)

    select_asset2.on_change('value', update_asset)
    select_inflation2.on_change('value', update_inflation)

    layout1 = row(select_asset2, select_inflation2)
    layout2 = row(p)

    doc.add_root(column(layout1, layout2))

In [99]:
show(plot_proportion, notebook_url="http://localhost:8888")